In [85]:
import arcpy
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,roc_curve,precision_recall_curve
from sklearn.model_selection import cross_val_score,train_test_split
from autogluon.tabular import TabularPredictor

In [86]:
# 分析函数
# 采样
def sample_point(point_,raster_,out_name):
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    TableToTable(table_,out_path,out_name)
    return None
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster
# 数组整形
def resize_arrays(A, B, fill_value=0):
    """调整数组形状一致"""
    new_shape = (max(A.shape[0], B.shape[0]), max(A.shape[1], B.shape[1]))

    if A.shape != new_shape:
        if A.shape[0] < new_shape[0]:
            padding_rows = new_shape[0] - A.shape[0]
            padding = np.full((padding_rows, A.shape[1]), fill_value)
            A = np.vstack((A, padding))
        elif A.shape[0] > new_shape[0]:
            A = A[:new_shape[0], :]

        if A.shape[1] < new_shape[1]:
            pad_width = ((0, 0), (0, new_shape[1] - A.shape[1]))
            A = np.pad(A, pad_width, mode='constant', constant_values=fill_value)
        elif A.shape[1] > new_shape[1]:
            A = A[:, :new_shape[1]]
    
    if B.shape != new_shape:
        if B.shape[0] < new_shape[0]:
            padding_rows = new_shape[0] - B.shape[0]
            padding = np.full((padding_rows, B.shape[1]), fill_value)
            B = np.vstack((B, padding))
        elif B.shape[0] > new_shape[0]:
            B = B[:new_shape[0], :]

        if B.shape[1] < new_shape[1]:
            pad_width = ((0, 0), (0, new_shape[1] - B.shape[1]))
            B = np.pad(B, pad_width, mode='constant', constant_values=fill_value)
        elif B.shape[1] > new_shape[1]:
            B = B[:, :new_shape[1]]
    
    return A, B
# rf寻找最优参数
def rf_best_param(X_train,y_train,n_estimators_range,k=5):
    """默认为5折交叉验证,评价指标为R2"""
    # 设置树的数目范围
    n_estimators_range = n_estimators_range
    cv_scores = []
    # 使用交叉验证
    for n_estimators in n_estimators_range:
        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
        scores = cross_val_score(rf,X_train, y_train, cv=k, scoring='r2')  # K折交叉验证
        cv_scores.append(scores.mean())
    # 选择最优数量的树
    optimal_n_estimators = n_estimators_range[cv_scores.index(max(cv_scores))]
    return optimal_n_estimators




In [101]:
# 地理数据库路径
base_gdb_5m = r"D:\ArcgisData\basedata\basetrain_5m.gdb"
base_gdb_30m = r"D:\ArcgisData\basedata\basetrain_30m.gdb"
# 数据点文件路径
point_data = r"D:\ArcgisData\pred_tl\pred_database\TL.gdb\SY_sample_pro"
# 存储采样数据表的文件地理数据库
sample_gdb_path = r"D:\ArcgisData\pred_organic_p_n\feature_table\tableresult.gdb"
# 存储采样结果CSV文件的路径
sample_csv = r"D:\ArcgisData\pred_organic_p_n\feature_table\feature_table_result"
# 输出CSV文件的名称
sample_csv_name = "feature_table_result.csv"
# 目标标签
target_label = "A"
# 随机森林树的范围
n_estimators_range = range(10, 2000, 300)
# rf模型存储路径
modle_save_path = r"D:\ArcgisData\pred_tl\pred_moudle"
# rf模型名称
modle_name = "rf_model.pkl"
# 栅格输出标准化的数据库
stander_raster_gdb = base_gdb_30m
# 标准化待预测数据分割路径
cut_csv_path = r"D:\ArcgisData\pred_tl\pred_table\cut_csv"
# 预测完成CSV文件存储路径
pred_csv_path = r"D:\ArcgisData\pred_tl\pred_table\pre_csv"
# 完整预测数据存储路径
merge_csv_path = r"D:\ArcgisData\pred_tl\pred_table\merge_csv"
# 完整预测数据存储名称
merge_csv_name = "merge.csv"
# 栅格输出预测结果数据库
pred_raster_gdb = r"D:\ArcgisData\pred_tl\pred_database\TL_basedata.gdb"
# 栅格输出预测结果栅格名称
pred_raster_name = "TL_pred_raster"

In [88]:
# 采样点数据名称
sample_name = 'SY_sample_pro'
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
elements_yes = ['OBJECTID', 'A', 'B','C','D']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)


['OBJECTID', 'Shape', '代码', 'Shape_Leng', 'A', 'B', 'C', 'D']
['OBJECTID', 'A', 'B', 'C', 'D']


In [89]:
# 指定指标
# 指标列表['TWI5','TPI201','TPI101','TPI11','TPI3','TMP','SOILQS','SLOP','PRE','NIGTH','NDVI','DEM','CUR','ASP','PLCUR','POCUR','OSJL','LAT','LON','DZ','DL']
feature_list = ['TWI5','TPI201','TMP','SLOP','PRE','NDVI','DEM','ASP','LAT','LON','DZ','DL']


In [90]:
# 使用训练点数据集采样并输出到csv文件
# 选择用于采样的数据库
env.workspace = base_gdb_5m # 切换工作空间用于采样
# 选择用于采样的要素类
point_data = point_data
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(sample_gdb_path)
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(os.path.dirname(sample_gdb_path), "tableresult", "CURRENT")
# 逐个采样并保存到csv文件
for one_feature in feature_list:
    sample_point(point_data,one_feature,os.path.join(sample_gdb_path,one_feature))
env.workspace = os.path.join(sample_gdb_path) # 切换工作空间用于导出csv文件
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
result_df.to_csv(os.path.join(sample_csv,sample_csv_name),index=False)

In [91]:
# 读取CSV文件使用RF训练模型
data = pd.read_csv(os.path.join(sample_csv,sample_csv_name))
print(len(data))
# 删除有缺失值的行
data.dropna(inplace=True)
len(data),data.columns
# 划分数据集
X = data[feature_list]
y = data[target_label]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
# 寻找最优树的个数
best_tree = rf_best_param(X_train,y_train,n_estimators_range,2)
print(f"最优树的个数为:{best_tree}")
# 初始化和训练随机森林模型
rf = RandomForestClassifier(n_estimators=best_tree, random_state=42)
rf.fit(X_train,y_train)

594
最优树的个数为:910


RandomForestClassifier(n_estimators=910, random_state=42)

In [92]:
# 保存模型
with open(os.path.join(modle_save_path,modle_name), 'wb') as f:
    pickle.dump(rf, f)
# 加载模型
with open(os.path.join(modle_save_path,modle_name), 'rb') as f:
    predictor = pickle.load(f)

In [93]:
# 试验区域预测数据统一
env.workspace = stander_raster_gdb
# 查看数据形状的一致性
stand_shape = arcpy.RasterToNumPyArray(feature_list[0]).shape
for one_raster in feature_list[1:]:
    if arcpy.RasterToNumPyArray(one_raster).shape != stand_shape:
        # 将形状不同的数据输出
        print(one_raster)
# 构造数据
feature_array_list = []
for one_raster in feature_list:
    # 读取数据
    one_array = arcpy.RasterToNumPyArray(one_raster)
    # 将数据转换为一维数组
    one_array = one_array.flatten()
    # 将数据添加到列表中
    feature_array_list.append(one_array)
# 将列表转换为数组
feature_array_list = np.column_stack(feature_array_list)

In [94]:
# 将数组转换为数据帧
feature_df = pd.DataFrame(feature_array_list,columns=feature_list)
# 修改部分列的数据类型
feature_df['DL'] = feature_df['DL'].astype('str')
feature_df['DZ'] = feature_df['DZ'].astype('str')

In [95]:
# 分割数据并保存为csv文件
chunk_size = 400000  # 每个文件的行数
total_rows = feature_df.shape[0]
for i in range(0, total_rows, chunk_size):
    start = i
    end = min(i + chunk_size, total_rows)
    filename =  os.path.join(cut_csv_path,f'data_chunk_{i}.csv') # 文件名格式可以根据您的需要进行修改
    df_chunk = feature_df.iloc[start:end]
    df_chunk.to_csv(filename, index=False)
    print(i)

0
400000
800000
1200000
1600000
2000000
2400000
2800000
3200000
3600000
4000000
4400000


In [96]:
# 读取数据
cut_csv_table_list = [os.path.join(cut_csv_path,_) for _ in os.listdir(cut_csv_path)]
# 排序
sorted_csv_files = sorted(cut_csv_table_list, key=lambda x: int(x.rsplit('_', 1)[-1].split('.')[0]))

In [97]:
# 预测试验区域数据
pre_n = 0  # 记录预测的表索引
for one_table in sorted_csv_files:
    data_df = pd.read_csv(one_table)
    temp_pred = predictor.predict(data_df)
    temp_pred = pd.DataFrame(temp_pred,columns=[target_label])
    temp_pred.to_csv(os.path.join(pred_csv_path,f"{pre_n}.csv"))
    pre_n += 1
    print(pre_n)

1
2
3
4
5
6
7
8
9
10
11
12


In [103]:
# 读取预测结果
pre_csv_list = [os.path.join(pred_csv_path,_) for _ in os.listdir(pred_csv_path)]
sorted_pre_csv_list = sorted(pre_csv_list,key=lambda x:int(x.rsplit('\\', -1)[-1].split('.')[0]))
# 保存完整的预测结果
pred_df = pd.read_csv(sorted_pre_csv_list[0])
for one_pred in sorted_pre_csv_list[1:]:
    temp_df = pd.read_csv(one_pred)
    pred_df = pd.concat([pred_df,temp_df],axis=0)
    print(one_pred)
pred_df.to_csv(os.path.join(merge_csv_path,merge_csv_name),index=False)

D:\ArcgisData\pred_tl\pred_table\pre_csv\1.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\2.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\3.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\4.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\5.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\6.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\7.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\8.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\9.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\10.csv
D:\ArcgisData\pred_tl\pred_table\pre_csv\11.csv


In [104]:
# 输出预测栅格图
env.workspace = stander_raster_gdb
pred_df['category_encoded'] = pd.factorize(pred_df[target_label])[0]
pred_df['category_encoded'] = pred_df['category_encoded'].astype('float32')
raster_array = np.reshape(pred_df['category_encoded'].values,arcpy.RasterToNumPyArray("DEM").shape)
env.extent = "DEM"  # 指定输出栅格的范围
pred_result_raster = mask_raster(raster_array, "DEM",30)
pred_result_raster.save(os.path.join(pred_raster_gdb,pred_raster_name))

In [ ]:
# 预览栅格图
arcpy.sa.Raster(os.path.join(pred_raster_gdb,pred_raster_name))